In [2]:
!pip show pydantic | grep Version

Version: 2.11.7


In [3]:
import pydantic
print(pydantic.__version__)

2.11.7


In [24]:
from pydantic import BaseModel, ValidationError
from datetime import datetime, UTC

class User(BaseModel):
    username: str
    email: str


In [5]:
user = User()
print(user)

ValidationError: 2 validation errors for User
username
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
email
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

Look for the Validation Error, it indicates 2 validation errors.

In [6]:
user = User(
    username='john_doe',
    email='not-an-email'
)
print(user)

username='john_doe' email='not-an-email'


In [7]:
user = User(
    username='john_doe',
    email='not-an-email',
    bio = 'Just a regular user.'
)
print(user)

username='john_doe' email='not-an-email'


In [9]:
# Modify the Class to include optional fields and default values
class User(BaseModel):
    username: str
    email: str

    # New optional fields with default values
    bio: str = ""
    is_active: bool = True


user = User(
    username='john_doe',
    email='not-an-email',
    bio = 'Just a regular user.'
)
print(user)

username='john_doe' email='not-an-email' bio='Just a regular user.' is_active=True


In [11]:
# Modify the Class to include optional fields and default values
class User(BaseModel):
    username: str
    email: str

    # New optional fields with default values
    bio: str = ""
    is_active: bool = True

    # New Optional field without default value
    full_name: str | None = None

user = User(
    username='john_doe',
    email='not-an-email',
    bio = 'Just a regular user.'
)
print(user)

username='john_doe' email='not-an-email' bio='Just a regular user.' is_active=True full_name=None


In [12]:
# Access field values
print("Username:", user.username)
print("Email:", user.email)
print("Bio:", user.bio)
print("Is Active:", user.is_active)
print("Full Name:", user.full_name)

Username: john_doe
Email: not-an-email
Bio: Just a regular user.
Is Active: True
Full Name: None


In [13]:
# Update the user field values
user.bio = "An updated bio."
print("Bio:", user.bio)

Bio: An updated bio.


In [14]:
# Update the user field values, demonstrating limitation
user.bio = 12345  # This will not raise a validation error
print("Bio:", user.bio)

Bio: 12345


In [16]:
# Revert the Bio field to valid value
user.bio = "Reverted to valid bio."
print("Bio:", user.bio)

Bio: Reverted to valid bio.


In [17]:
# Convert the model to a dictionary
user_dict = user.model_dump()
print("User as dict:", user_dict)

User as dict: {'username': 'john_doe', 'email': 'not-an-email', 'bio': 'Reverted to valid bio.', 'is_active': True, 'full_name': None}


In [20]:
# Convert to JSON
user_json = user.model_dump_json()
print("User as JSON:", user_json)

User as JSON: {"username":"john_doe","email":"not-an-email","bio":"Reverted to valid bio.","is_active":true,"full_name":null}


In [21]:
user_json = user.model_dump_json(indent=2)
print("User as JSON:", user_json)

User as JSON: {
  "username": "john_doe",
  "email": "not-an-email",
  "bio": "Reverted to valid bio.",
  "is_active": true,
  "full_name": null
}


In [23]:
# using ValidationError to handle errors
try:
    user2 = User(
        username=None,
        email=12345
    )
except ValidationError as e:
    print("Validation error occurred:")
    print(e)
else:
    print(user2)


Validation error occurred:
2 validation errors for User
username
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
email
  Input should be a valid string [type=string_type, input_value=12345, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


In [26]:
from pydantic import Field
from typing import Literal

class BlogPost(BaseModel):
    title: str
    content: str
    view_count: int = 0
    is_published: bool = False
    published_at: datetime | None = None

    tags: list[str] = Field(default_factory=list)
    
    # created_at: datetime = datetime.now(UTC)
    # Or using default_factory for dynamic default value
    # created_at: datetime = Field(default_factory=datetime.now(tz=UTC))
    # Or using default_factory for dynamic default value
    created_at: datetime = Field(default_factory=lambda: datetime.now(tz=UTC))

    # Union Type Example to hold multiple types for a field as str and int
    author_id : str | int

    status: Literal['draft', 'published', 'archived'] = 'draft'


In [28]:
# New blog post instance
post = BlogPost(
    title="My First Blog Post",
    content="This is the content of my first blog post.",
    # author_id=101
    author_id="author_101"
)
print(post)

title='My First Blog Post' content='This is the content of my first blog post.' view_count=0 is_published=False published_at=None tags=[] created_at=datetime.datetime(2025, 11, 2, 6, 40, 26, 273697, tzinfo=datetime.timezone.utc) author_id='author_101' status='draft'


In [30]:
# Adding Constrained Types, for this will use Annotated
from typing import Annotated

class User(BaseModel):
    uid: Annotated[int, Field(gt=0)]  # uid must be greater than 0
    username: Annotated[str, Field(min_length=3, max_length=50)]  # username length constraints
    age: Annotated[int, Field(ge=13, le=120)]  # age must be greater or equal (ge) to 13 and less than equal to 120
    email: str
    verified_at: datetime | None = None
    bio: str = ""
    is_active: bool = True
    full_name: str | None = None


class BlogPost(BaseModel):
    title: Annotated[str, Field(min_length=5, max_length=200)]
    content: Annotated[str, Field(min_length=20)]
    author_id : str | int
    view_count: Annotated[int, Field(ge=0)] = 0
    is_published: bool = False
    published_at: datetime | None = None
    tags: list[str] = Field(default_factory=list)
    created_at: datetime = Field(default_factory=lambda: datetime.now(tz=UTC))
    
    status: Literal['draft', 'published', 'archived'] = 'draft'

    slug: Annotated[str, Field(pattern=r"^[a-z0-9-]+$")]

In [31]:
# Invalid User instance to demonstrate constraints
try:
    invalid_user = User(
        uid=-1,
        username="ab",
        age=10,
        email="invalid-email"
    )
except ValidationError as e:
    print("Validation error for invalid user:")
    print(e)
else:
    print(invalid_user)

Validation error for invalid user:
3 validation errors for User
uid
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than
username
  String should have at least 3 characters [type=string_too_short, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
age
  Input should be greater than or equal to 13 [type=greater_than_equal, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal


In [35]:
# Email validation example
from pydantic import EmailStr, HttpUrl, SecretStr
from uuid import UUID, uuid4

In [36]:
class User(BaseModel):
    uid: UUID = Field(default_factory=uuid4)
    username: Annotated[str, Field(min_length=3, max_length=50)]  # username length constraints
    age: Annotated[int, Field(ge=13, le=120)]  # age must be greater or equal (ge) to 13 and less than equal to 120
    email: EmailStr
    verified_at: datetime | None = None
    bio: str = ""
    is_active: bool = True
    full_name: str | None = None
    website: HttpUrl | None = None
    password: SecretStr
    

In [37]:
# Valid User instance
user = User(
    username="valid_user",
    age=25,
    email="abcd@gmail.com",
    website="https://example.com",
    password="strong_password_123"
)
print(user)

uid=UUID('34c9b067-03cd-4b42-bf3f-a8e6b60457d2') username='valid_user' age=25 email='abcd@gmail.com' verified_at=None bio='' is_active=True full_name=None website=HttpUrl('https://example.com/') password=SecretStr('**********')


In [39]:
print(user.password)
print(user.password.get_secret_value())

**********
strong_password_123


# Custom Validators using decorator's

In [40]:
from pydantic import field_validator, model_validator, ValidationInfo

In [41]:
class User(BaseModel):
    uid: UUID = Field(default_factory=uuid4)
    username: Annotated[str, Field(min_length=3, max_length=50)]  # username length constraints
    age: Annotated[int, Field(ge=13, le=120)]  # age must be greater or equal (ge) to 13 and less than equal to 120
    email: EmailStr
    verified_at: datetime | None = None
    bio: str = ""
    is_active: bool = True
    full_name: str | None = None
    website: HttpUrl | None = None
    password: SecretStr

    @field_validator("username")
    @classmethod
    def validate_username(cls, v: str) -> str:
        if not v.replace("_", "").isalnum():
            raise ValueError("Username must be aplhanumeric (underscores allowed)")
        return v.lower()
    

In [43]:
# Valid User instance
user = User(
    username="Mayank_TriPathi",
    age=25,
    email="abcd@gmail.com",
    website="https://example.com",
    password="strong_password_123"
)
print(user.username)

mayank_tripathi


In [46]:
class User(BaseModel):
    uid: UUID = Field(default_factory=uuid4)
    username: Annotated[str, Field(min_length=3, max_length=50)]  # username length constraints
    age: Annotated[int, Field(ge=13, le=120)]  # age must be greater or equal (ge) to 13 and less than equal to 120
    email: EmailStr
    verified_at: datetime | None = None
    bio: str = ""
    is_active: bool = True
    full_name: str | None = None
    website: HttpUrl | None = None
    password: SecretStr

    @field_validator("username")
    @classmethod
    def validate_username(cls, v: str) -> str:
        if not v.replace("_", "").isalnum():
            raise ValueError("Username must be aplhanumeric (underscores allowed)")
        return v.lower()
    
    @field_validator("website", mode="before")
    @classmethod
    def validate_website(cls, v: str | None) -> str | None:
        if v and not v.startswith(("https://", "http://")):
            # raise ValueError("Website URL must start with 'https://'")
            return f"https://{v}"
        return v

In [48]:
user = User(
    username="Mayank_TriPathi",
    age=25,
    email="abcd@gmail.com",
    website="example.com",
    password="strong_password_123"
)
print(user)

uid=UUID('3f048ddd-35f5-4973-a107-cf0bca77a26e') username='mayank_tripathi' age=25 email='abcd@gmail.com' verified_at=None bio='' is_active=True full_name=None website=HttpUrl('https://example.com/') password=SecretStr('**********')


In [ ]:
# Model Validator



In [54]:
# Model Config
from pydantic import ConfigDict

class User(BaseModel):
    model_config = ConfigDict(
        populate_by_name=True,

    )
    uid: UUID = Field(alias="id", default_factory=uuid4)
    username: Annotated[str, Field(min_length=3, max_length=50)]  # username length constraints
    age: Annotated[int, Field(ge=13, le=120)]  # age must be greater or equal (ge) to 13 and less than equal to 120
    email: EmailStr
    password: SecretStr

In [55]:
user_data = {
    "id": "3f048ddd-35f5-4973-a107-cf0bca77a26e",
    "username": "Mayank",
    "email": "abe@example.com",
    "age": 39,
    "password": "somesecret123"
}

user = User.model_validate(user_data)
print(user.model_dump_json(indent=2))

{
  "uid": "3f048ddd-35f5-4973-a107-cf0bca77a26e",
  "username": "Mayank",
  "age": 39,
  "email": "abe@example.com",
  "password": "**********"
}


In [56]:
print(user.model_dump_json(indent=2, by_alias=True))

{
  "id": "3f048ddd-35f5-4973-a107-cf0bca77a26e",
  "username": "Mayank",
  "age": 39,
  "email": "abe@example.com",
  "password": "**********"
}


In [57]:
print(user.model_dump_json(indent=2, by_alias=True, exclude={"password"}))

{
  "id": "3f048ddd-35f5-4973-a107-cf0bca77a26e",
  "username": "Mayank",
  "age": 39,
  "email": "abe@example.com"
}


In [58]:
print(user.model_dump_json(indent=2, by_alias=True, include={"username", "email"}))

{
  "username": "Mayank",
  "email": "abe@example.com"
}


Other options in ConfigDict

```python
model_config = ConfigDict(
        populate_by_name=True,
        strict=True,
        extra="allow" ,
        validate_assignment=True, 
    )

```